# JavaScript in Jupyter #

## <font color="green">Special JavaScript addendum to core Python lecture series</font> ##

![js_logo](https://www.freepnglogos.com/uploads/javascript-png/javascript-logo-transparent-logo-javascript-images-3.png) 

### Nick Day 

### January 2023


## Running JavaScript in Jupyter ##

- Third party kernels are available for many languages and flavours of these.
- Kernels are available for Java (several options), R, Julia, Matlab, Octave, Scheme, Processing, Scala and many more languages
- Once installed, kernels may be switched at run time from the notebook Kernel\Change kernel menu
- Here we install the SciJava kernel through Anconda (https://anaconda.org/conda-forge/scijava-jupyter-kernel)
    - Open Anaconda
    - Run CMD.exe
    - Enter "**<font color="red">conda install -c conda-forge/label/cf201901 scijava-jupyter-kernel</font>**"
    - Open a Jupyter notebook and specify kernel before or select afterwards
    - If all goes well it will work out of-the-box
    - Because 'Groovy' is a polyglot kernel offering support for many languages there may be incompatibilities
     - ... including with various libraries 
     - ... and often with the 'main' method
     - ... with core Java code, 'main(String[] args)' issues may be resolved by removing the class wrapping the main and also the main method
     - ... see examples below


https://jakevdp.github.io/blog/2013/06/01/ipython-notebook-javascript-python-communication/

In [7]:
print("hello")

hello


In [18]:
%%javascript

// Function that accepts a string of code
var create_and_execute_cell_below = function (code){
    var nb = Jupyter.notebook

    // create cell below this one
    nb.insert_cell_below()

    // select cell below (the one we have created)
    var cell = nb.select_next().get_selected_cell()

    // set text in the cell
    cell.set_text(code)

    // execute cell
    cell.execute()
}

// run the function created above with code 'k = 1'
// and it will create a new cell, filled with 'k = 1'
// and it will execute that cell.
// if you run this cell using [ctrl] + [enter] only one cell
// will be created.
// if you run this cell using [Shift] + [enter] two cells
// will be created, the one of the [Shift] + [enter] command
// and the one of the function created above with the code.
create_and_execute_cell_below('k = 1')

<IPython.core.display.Javascript object>

In [1]:
from IPython.display import HTML

var kernel = IPython.notebook.kernel;
kernel.execute(command);

SyntaxError: invalid syntax (3449246251.py, line 3)

In [8]:
from IPython.display import HTML

input_form = """
<div style="background-color:gainsboro; border:solid black; width:300px; padding:20px;">
Variable Name: <input type="text" id="var_name" value="foo"><br>
Variable Value: <input type="text" id="var_value" value="bar"><br>
<button onclick="set_value()">Set Value</button>
</div>
"""

javascript = """
<script type="text/Javascript">
    function set_value(){
        var var_name = document.getElementById('var_name').value;
        var var_value = document.getElementById('var_value').value;
        var command = var_name + " = '" + var_value + "'";
        console.log("Executing Command: " + command);
        
        var kernel = IPython.notebook.kernel;
        kernel.execute(command);
    }
</script>
"""
HTML(input_form + javascript)

In [11]:
var kernel = IPython.notebook.kernel;
function callback(out_type, out_data)
{
    // do_something
}
kernel.execute(command, {"output": callback});

SyntaxError: invalid syntax (2326933582.py, line 1)

In [12]:
# Add an input form similar to what we saw above
input_form = """
<div style="background-color:gainsboro; border:solid black; width:600px; padding:20px;">
Code: <input type="text" id="code_input" size="50" height="2" value="sin(pi / 2)"><br>
Result: <input type="text" id="result_output" size="50" value="1.0"><br>
<button onclick="exec_code()">Execute</button>
</div>
"""

# here the javascript has a function to execute the code
# within the input box, and a callback to handle the output.
javascript = """
<script type="text/Javascript">
    function handle_output(out_type, out){
        console.log(out_type);
        console.log(out);
        var res = null;
         // if output is a print statement
        if(out_type == "stream"){
            res = out.data;
        }
        // if output is a python object
        else if(out_type === "pyout"){
            res = out.data["text/plain"];
        }
        // if output is a python error
        else if(out_type == "pyerr"){
            res = out.ename + ": " + out.evalue;
        }
        // if output is something we haven't thought of
        else{
            res = "[out type not implemented]";   
        }
        document.getElementById("result_output").value = res;
    }
    
    function exec_code(){
        var code_input = document.getElementById('code_input').value;
        var kernel = IPython.notebook.kernel;
        var callbacks = {'output' : handle_output};
        document.getElementById("result_output").value = "";  // clear output box
        var msg_id = kernel.execute(code_input, callbacks, {silent:false});
        console.log("button pressed");
    }
</script>
"""

HTML(input_form + javascript)

In [13]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [14]:
from IPython.display import HTML
from cStringIO import StringIO

# We'll use HTML to create a control panel with an
# empty image and a number of navigation buttons.

disp_html = """
<div class="animation" align="center">
<img id="anim_frame" src=""><br>
<button onclick="prevFrame()">Prev Frame</button>
<button onclick="reverse()">Reverse</button>
<button onclick="pause()">Pause</button>
<button onclick="play()">Play</button>
<button onclick="nextFrame()">Next Frame</button>
</div>
"""

# now the javascript to drive it.  The nextFrame() and prevFrame()
# functions will call the kernel and pull-down the frame which
# is generated.  The play() and reverse() functions use timeouts
# to repeatedly call nextFrame() and prevFrame().

javascript = """
<script type="text/Javascript">
var count = -1;  // keep track of frame number
var animating = 0;  // keep track of animation direction
var timer = null;
var kernel = IPython.notebook.kernel;

function output(out_type, out){
    data = out.data["text/plain"];
    document.getElementById("anim_frame").src = data.substring(1, data.length - 1);
    
    if(animating > 0){
        timer = setTimeout(nextFrame, 0);
    }
    else if(animating < 0){
        timer = setTimeout(prevFrame, 0);
    }
}

var callbacks = {'output' : output};

function pause(){
    animating = 0;
    if(timer){
        clearInterval(timer);
        timer = null;
    }
}

function play(){
    pause();
    animating = +1;
    nextFrame();
}

function reverse(){
    pause();
    animating = -1;
    prevFrame();
}

function nextFrame(){
    count += 1;
    var msg_id = kernel.execute("disp._get_frame_data(" + count + ")", callbacks, {silent:false});
}

function prevFrame(){
    count -= 1;
    var msg_id = kernel.execute("disp._get_frame_data(" + count + ")", callbacks, {silent:false});
}

// display the first frame
setTimeout(nextFrame, 0);

</script>
"""

# Here we create a class whose HTML representation is the above
# HTML and javascript.  Note that we've hard-coded the global
# variable name `disp` in the Javascript, so you'll have to assign
# the resulting object to this name in order to view it.

class DisplayAnimation(object):
    def __init__(self, anim):
        self.anim = anim
        self.fig = anim._fig
        plt.close(self.fig)
        
    def _get_frame_data(self, i):
        anim._draw_frame(i)
        buffer = StringIO()
        fig.savefig(buffer, format='png')
        buffer.reset()
        data = buffer.read().encode('base64')
        return "data:image/png;base64,{0}".format(data.replace('\n', ''))
    
    def _repr_html_(self):
        return disp_html + javascript

ModuleNotFoundError: No module named 'cStringIO'

###### References & Learning Resources#

- Gamma, E., Helm, R., Johnson, R., & Vlissides, J. (1995). Gamma, E., Helm, R., Johnson, R., & Vlissides, J. Mar. 1995. Design Patterns–Elements of Reusable Object-Oriented Software. Addison-Wesley.
- Jungwoo Ryoo (2015) Python: Design Patterns. Available at: https://www.linkedin.com/learning/python-design-patterns and https://www.lynda.com/Python-tutorials/Design-Patterns-Python/369187-2.html
- Phillips D. (2015) Python 3 Object-oriented Programming - Second Edition. PACKT Publishing.
- Shalloway, A., & Trott, J. R. (2002). Design patterns explained: A new perspective on object-oriented design. Addison-Wesley
- Tutorialspoint (undated) Design Patterns - Command Pattern. Available at https://www.tutorialspoint.com/design_pattern/command_pattern.htm 
- Tutorialspoint (undated) Design Patterns - Proxy Pattern. Available at https://www.tutorialspoint.com/design_pattern/proxy_pattern.htm
